In [27]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import yfinance as yf
from panel.interact import interact
import plotly.express as px
import panel as pn
pn.extension('plotly')
import seaborn as sns

In [128]:
ticker = "ADSK"

In [129]:
df_data = yf.download(ticker,
start="2020-10-01",
end="2021-04-14",
progress=False,
interval = "1d",
index_col = "Date",
infer_datetime_format= True,
parse_dates = True,                    
groupby = "column"
                )

In [130]:
signals_df = df_data.drop(columns = ["Open","Adj Close","Volume"])

In [131]:
from talib import ATR
def ATR_Trailing_Stop (high, low, close, period, factor):
    
    atr_val = ATR(high, low, close, timeperiod=period)
    trail = atr_val * factor
    
    prev = close[period] - trail[period]
    
    for i in range(period+1,trail.size):
        if (close[i] > prev) and (close[i-1] > prev):
            trail[i] = max(prev, close[i] - trail[i])
        elif (close[i] < prev) and (close[i-1] < prev):
            trail[i] = min(prev, close[i] + trail[i])
        elif close[i] > prev:
            trail[i] = close[i] - trail[i]
        else:
            trail[i] = close[i] + trail[i]
        prev = trail[i]
    return trail


In [132]:
def calc_signal (buy, sell):
    position = 0
    signals = pd.Series(index = buy.index, dtype = float)
    
    for i in range(0, signals.size):
        if position == 0 and buy[i] == 1:
            signals[i] = 1
            position = 1
        elif position == 1 and sell[i] == 1:
            signals[i] = -1
            position = 0
        else:
            signals[i] = 0
        
    return signals

In [133]:
from talib import ATR

#Define Window 13,34,89 Fibonanci numbers
EMAshort = 13
EMAmid = 34
EMAlong = 89

#Generate EMA
signals_df["EMAshort"] = signals_df["Close"].ewm(halflife=EMAshort).mean()
signals_df["EMAmid"] = signals_df["Close"].ewm(halflife=EMAmid).mean()
signals_df["EMAlong"] = signals_df["Close"].ewm(halflife=EMAlong).mean()
signals_df["Entry"] = 0.0
signals_df["Exit"] = 0.0

#Generate ATR Trailing Stop
close = signals_df.Close
high = signals_df.High
low = signals_df.Low
signals_df["ATRts"] = ATR_Trailing_Stop (high, low, close, 89, 10)

# Generate the trading signal
signals_df["Entry"][EMAshort:] = np.where(
    signals_df["EMAmid"][EMAshort:] - signals_df["EMAlong"][EMAshort:] > 0, 1.0, 0.0
)
signals_df["Exit"][EMAshort:] = np.where(
    signals_df["ATRts"][EMAshort:] - signals_df["Close"][EMAshort:] > 0, 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy"] = signals_df["Entry"].diff()
signals_df["Sell"] = signals_df["Exit"].diff()
signals_df["Entry/Exit"] = calc_signal (signals_df["Buy"], signals_df["Sell"])
signals_df.head()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,Sell,Entry/Exit
Date,,,,,,,,,,,,
2020-10-01,238.610001,232.750000,233.960007,233.960007,233.960007,233.960007,0.0,0.0,NaN,NaN,NaN,0.0
2020-10-02,231.139999,223.509995,225.559998,229.648059,229.717192,229.743647,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-05,230.130005,225.039993,228.929993,229.395834,229.449425,229.470314,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-06,231.570007,226.500000,228.009995,229.021192,229.078489,229.100958,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-07,234.210007,229.309998,233.009995,229.906208,229.897173,229.894990,0.0,0.0,NaN,0.0,0.0,0.0


In [134]:
def calc_holdings (EntryExit):
    holding = 0
    signals = pd.Series(index = EntryExit.index, dtype = float)
    
    for i in range(0, EntryExit.size):
        holding = holding + EntryExit[i]
        signals[i] = holding
        
    return signals

In [135]:
signals_df['holding'] = calc_holdings(signals_df["Entry/Exit"])

In [136]:
# Set initial capital
initial_capital = signals_df[signals_df['Entry/Exit']==1].iloc[0]['Close']*1000

# Set the share size
share_size = 1000

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['holding']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,Sell,Entry/Exit,holding,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,
2021-04-07,287.589996,283.239990,286.119995,278.318341,280.922364,278.937427,1.0,0.0,225.228416,0.0,0.0,0.0,1.0,1000.0,0.0,286119.995117,0.0,286119.995117,0.002172,0.111836
2021-04-08,297.760010,288.649994,293.429993,279.103742,281.193954,279.114019,1.0,0.0,225.228416,0.0,0.0,0.0,1.0,1000.0,0.0,293429.992676,0.0,293429.992676,0.025549,0.140242
2021-04-09,297.899994,290.380005,297.570007,280.063445,281.548999,279.337915,1.0,0.0,225.228416,0.0,0.0,0.0,1.0,1000.0,0.0,297570.007324,0.0,297570.007324,0.014109,0.156330
2021-04-12,296.250000,293.000000,294.500000,280.813685,281.829365,279.521050,1.0,0.0,225.228416,0.0,0.0,0.0,1.0,1000.0,0.0,294500.000000,0.0,294500.000000,-0.010317,0.144400
2021-04-13,298.660004,293.119995,295.350006,281.569076,282.121635,279.711418,1.0,0.0,225.228416,0.0,0.0,0.0,1.0,1000.0,0.0,295350.006104,0.0,295350.006104,0.002886,0.147703


In [137]:
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)



In [138]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': ticker,
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [139]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize Indicators
indicators = signals_df[['EMAmid', 'EMAlong','ATRts']].hvplot(
    ylabel='Price in $',
    width=750,
    height=300
)


# Overlay plots
entry_exit_plot = security_close * indicators * entry * exit
entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [140]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(title=f"{ticker}: EMA & ATR Trailing Stop Portfolio Value", xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [141]:
# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["SMASignal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["SMASignal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["SMAEntry/Exit"] = signals_df["SMASignal"].diff()

In [142]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['SMA Position'] = share_size * signals_df['SMASignal']

# Find the points in time where a 500 share position is bought or sold
signals_df['SMA Entry/Exit Position'] = signals_df['SMA Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['SMA Portfolio Holdings'] = signals_df['Close'] * signals_df['SMA Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['SMA Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['SMA Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['SMA Portfolio Total'] = signals_df['SMA Portfolio Cash'] + signals_df['SMA Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['SMA Portfolio Daily Returns'] = signals_df['SMA Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['SMA Portfolio Cumulative Returns'] = (1 + signals_df['SMA Portfolio Daily Returns']).cumprod() - 1


In [143]:
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Algo1 Backtest','SMA Backtest']

portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

In [144]:
portfolio_evaluation_df.loc['Cumulative Returns']['Algo1 Backtest'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df.loc['Annual Return']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Volatility']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Sharpe Ratio']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].mean() * 252) / (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))

portfolio_evaluation_df.loc['Cumulative Returns']['SMA Backtest'] = signals_df['SMA Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df.loc['Annual Return']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Volatility']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Sharpe Ratio']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].mean() * 252) / (signals_df['SMA Portfolio Daily Returns'].std() * np.sqrt(252))

#Algo1 Sortino Ratio
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

#SMA Sortino Ratio
SMA_sortino_ratio_df = signals_df[['SMA Portfolio Daily Returns']].copy()
SMA_sortino_ratio_df.loc[:,'SMA Downside Returns'] = 0

#Algo1 Downside Returns
target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

#SMA Downside Returns
SMA_mask = SMA_sortino_ratio_df['SMA Portfolio Daily Returns'] < target
SMA_sortino_ratio_df.loc[mask, 'SMA Downside Returns'] = SMA_sortino_ratio_df['SMA Portfolio Daily Returns']**2

#Also1 Sortino Raio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev
portfolio_evaluation_df.loc['Sortino Ratio']['Algo1 Backtest'] = sortino_ratio

#SMA Sortino Raio
SMA_down_stdev = np.sqrt(SMA_sortino_ratio_df['SMA Downside Returns'].mean()) * np.sqrt(252)
SMA_expected_return = SMA_sortino_ratio_df['SMA Portfolio Daily Returns'].mean() * 252
SMA_sortino_ratio = SMA_expected_return/SMA_down_stdev
portfolio_evaluation_df.loc['Sortino Ratio']['SMA Backtest'] = SMA_sortino_ratio

portfolio_evaluation_df

,Algo1 Backtest,SMA Backtest
Annual Return,0.319155,-0.146687
Cumulative Returns,0.147703,-0.0845574
Annual Volatility,0.329045,0.215438
Sharpe Ratio,0.969944,-0.680878
Sortino Ratio,1.38963,-0.898575


In [145]:
# Visualize exit position relative to close price
SMAexit = signals_df[signals_df['SMAEntry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize entry position relative to close price
SMAentry = signals_df[signals_df['SMAEntry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize close price for the investment
SMAsecurity_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize moving averages
SMAmoving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=750,
    height=300
)

# Overlay plots
SMAentry_exit_plot = SMAsecurity_close * SMAmoving_avgs * SMAentry * SMAexit
SMAentry_exit_plot.opts(title=f'{ticker}: SMA Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [146]:
# Visualize exit position relative to total portfolio value
SMAexit = signals_df[signals_df['SMAEntry/Exit'] == -1.0]['SMA Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize entry position relative to total portfolio value
SMAentry = signals_df[signals_df['SMAEntry/Exit'] == 1.0]['SMA Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize total portoflio value for the investment
SMAtotal_portfolio_value = signals_df[['SMA Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Overlay plots
SMAportfolio_entry_exit_plot = SMAtotal_portfolio_value * SMAentry * SMAexit
SMAportfolio_entry_exit_plot.opts(title=f'{ticker}: SMA Portfolio Value',xaxis=None)

:Overlay
   .Curve.I                        :Curve   [Date]   (SMA Portfolio Total)
   .Scatter.SMA_Portfolio_Total.I  :Scatter   [Date]   (SMA Portfolio Total)
   .Scatter.SMA_Portfolio_Total.II :Scatter   [Date]   (SMA Portfolio Total)

In [147]:
signals_df.head()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,...,SMA100,SMASignal,SMAEntry/Exit,SMA Position,SMA Entry/Exit Position,SMA Portfolio Holdings,SMA Portfolio Cash,SMA Portfolio Total,SMA Portfolio Daily Returns,SMA Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01,238.610001,232.750000,233.960007,233.960007,233.960007,233.960007,0.0,0.0,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,231.139999,223.509995,225.559998,229.648059,229.717192,229.743647,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,257339.996338,257339.996338,NaN,NaN
2020-10-05,230.130005,225.039993,228.929993,229.395834,229.449425,229.470314,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,257339.996338,257339.996338,0.0,0.0
2020-10-06,231.570007,226.500000,228.009995,229.021192,229.078489,229.100958,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,257339.996338,257339.996338,0.0,0.0
2020-10-07,234.210007,229.309998,233.009995,229.906208,229.897173,229.894990,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,257339.996338,257339.996338,0.0,0.0


In [148]:
SMA_trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [149]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['SMAEntry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['SMA Portfolio Holdings']
        share_size = row['SMA Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['SMAEntry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['SMA Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        SMA_trade_evaluation_df = SMA_trade_evaluation_df.append(
            {
                'Stock': ticker,
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
SMA_trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,ADSK,2021-02-24,2021-03-31,1000.0,298.910004,277.149994,298910.003662,277149.993896,-21760.009766


In [150]:
entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [151]:
portfolio_analysis = pn.Column("#Entry/Exit Comparison",
                                pn.Row(
                                   entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None), 
                                   SMAentry_exit_plot.opts(title=f'{ticker}: SMA Entry/Exit', xaxis=None)),
                                "#Portfolio Value Comparison",
                               pn.Row(
                                   portfolio_entry_exit_plot.opts(title=f"{ticker}: EMA & ATR Trailing Stop Portfolio Value", xaxis=None),
                                   SMAportfolio_entry_exit_plot.opts(title=f'{ticker}: SMA Portfolio Value',xaxis=None)),
                               portfolio_evaluation_df

)

In [152]:
portfolio_analysis.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] HoloViews(Overlay)
        [1] HoloViews(Overlay)
    [2] Markdown(str)
    [3] Row
        [0] HoloViews(Overlay)
        [1] HoloViews(Overlay)
    [4] DataFrame(DataFrame)